In [1]:
%matplotlib inline
import torch
from torch.utils.data import DataLoader, Subset, default_collate
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision.transforms import v2
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import timm
from timm.models.swin_transformer import SwinTransformer
import numpy as np
from torch.utils.tensorboard import SummaryWriter

import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import logging

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Define transformations

In [2]:
cutmix = v2.CutMix(num_classes=100)
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)


def collate_fn_cutmix(batch):
    return cutmix(*default_collate(batch))


def prepare_datasets(train_transform, test_transform, cutmix_, batch_size):
    trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
    testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)
    
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

    if cutmix_ == 0:
        trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
        testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)
    elif cutmix_ == 1:
        trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=collate_fn_cutmix)
        testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=collate_fn_cutmix)

    return trainloader, testloader


# Define transformations
test_transform = v2.Compose([
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),  # to float32 in [0, 1]
    v2.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761]),
])

train_transform_basic = v2.Compose([
    v2.PILToTensor(),
    v2.RandomResizedCrop(size=(32, 32), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),  # to float32 in [0, 1]
    v2.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761]),
])

## Training function

In [3]:
def setup_logger(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    logging.basicConfig(
        filename=os.path.join(log_dir, 'training.log'),
        level=logging.INFO,
        format='%(asctime)s %(levelname)s: %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        filemode='w'
    )
    logger = logging.getLogger('train_model_logger')
    return logger

def train_model(model, trainloader, testloader, optimizer, num_epochs, scheduler=None, warmup_epochs=10, log_dir='./logs'):
    writer = SummaryWriter(log_dir=log_dir)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    save_dir = log_dir

    # Set up logging with timestamp
    logger = setup_logger(log_dir)

    model = model.to(device)
    train_losses = []
    val_losses = []
    val_accuracies = []
    best_val_acc = 0.0

    pbar_epochs = tqdm(total=num_epochs, desc='Overall Progress', position=0)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        if epoch < warmup_epochs:
            lr_scale = min(1., float(epoch + 1) / warmup_epochs)
            for pg in optimizer.param_groups:
                pg['lr'] = lr_scale * pg['initial_lr']
        
        pbar_batches = tqdm(total=len(trainloader), desc=f'Epoch {epoch+1}/{num_epochs}', position=1, leave=False)
        
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.3)
            
            
            optimizer.step()

            # Track statistics
            running_loss += loss.item() * inputs.size(0)
            pbar_batches.set_postfix(loss=loss.item())
            pbar_batches.update()

        pbar_batches.close()
        epoch_train_loss = running_loss / len(trainloader.dataset)
        train_losses.append(epoch_train_loss)

        # Write to TensorBoard
        writer.add_scalar('train_loss', epoch_train_loss, epoch)
        
        if scheduler is not None:
            scheduler.step()
        
        # Validation loop
        model.eval()
        correct_val = 0
        total_val = 0
        running_val_loss = 0.0

        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                if labels.ndim == 2:
                    labels = labels.argmax(dim=1)
                correct_val += (predicted == labels).sum().item()

        # Calculate average validation loss and accuracy
        epoch_val_loss = running_val_loss / len(testloader.dataset)
        epoch_val_acc = correct_val / total_val
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        # Write to TensorBoard
        writer.add_scalar('validation_loss', epoch_val_loss, epoch)
        writer.add_scalar('validation_accuracy', epoch_val_acc, epoch)

        pbar_epochs.set_postfix(
                train_loss=f"{epoch_train_loss:.4f}", 
                val_loss=f"{epoch_val_loss:.4f}", 
                val_acc=f"{100*epoch_val_acc:.2f}%"
            )
        pbar_epochs.update()
        
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            print("Saving...")
            torch.save(model.state_dict(), os.path.join(save_dir, f'best_model_ckpt.pth'))
            logger.info(f"Epoch [{epoch+1}/{num_epochs}], New best validation accuracy: {100*epoch_val_acc:.2f}% - Model saved.")

        # Logging epoch information
        logger.info(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {100*epoch_val_acc:.2f}%")

    pbar_epochs.close()
    writer.close()
    print('Training complete!')
    print(f'Best Validation Accuracy: {best_val_acc:.4f}')
    logger.info(f'Training complete! Best Validation Accuracy: {100*best_val_acc:.2f}%')

    return train_losses, val_losses, val_accuracies


## Model definition

In [4]:
class ResNet50_CIFAR100(torch.nn.Module):
    def __init__(self):
        super(ResNet50_CIFAR100, self).__init__()
        self.model = models.resnet50(weights=None, num_classes=100)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

    def forward(self, x):
        return self.model(x)
    
class SwinViT_T_CIFAR100(nn.Module):
    def __init__(self, num_classes=100):
        super(SwinViT_T_CIFAR100, self).__init__()
        self.swin = SwinTransformer(
            img_size=32,
            patch_size=2,
            num_classes=num_classes,
            embed_dim=96,
            depths=(2, 2, 6, 2),
            num_heads=(3, 6, 12, 24),
            window_size=4,
        )

    def forward(self, x):
        x = self.swin(x)
        return x

In [32]:
def count_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [33]:
count_parameters(SwinViT_T_CIFAR100())

27574318

In [28]:
count_parameters(ResNet50_CIFAR100())

23705252

## Try different configurations

In [21]:
model_name = "ResNet50_CIFAR100"
num_epochs = 20
b_lr_wd = [
    (128, 0.005, 1e-5),
    (128, 0.005, 1e-4),
    (128, 0.005, 1e-3),
    (128, 0.001, 1e-3),
    (128, 0.001, 1e-4),
    (128, 0.001, 1e-5),
    (256, 0.005, 1e-5),
    (256, 0.005, 1e-4),
    (256, 0.005, 1e-3),
    (256, 0.001, 1e-5),
    (256, 0.001, 1e-4),
    (256, 0.001, 1e-3),
]
optimizer_name = 'SGD'

for batch_size, lr, weight_decay in b_lr_wd:
    trainloader_cutmix, testloader_cutmix = prepare_datasets(train_transform_basic, test_transform, cutmix_=1, batch_size=batch_size)
    model = ResNet50_CIFAR100()
    model = model.to(device)
    print(f" ======> Training {model_name}: {optimizer_name}-{batch_size}_lr={lr}_weightdecay={weight_decay}")
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.97)
    log_dir = f'./official_models_tuning/{model_name}_cutmix/{optimizer_name}-{batch_size}_lr{lr}_wd-{weight_decay}_e{num_epochs}'
    train_model(model, trainloader_cutmix, testloader_cutmix, optimizer, num_epochs, scheduler=scheduler, log_dir=log_dir)

Files already downloaded and verified
Files already downloaded and verified
 ======> Training ResNet50_CIFAR100: SGD-128_lr=0.005_weightdecay=1e-05


Overall Progress: 100%|██████████| 20/20 [07:19<00:00, 21.96s/it, train_loss=3.8248, val_acc=16.56%, val_loss=3.8279]


Training complete!
Best Validation Accuracy: 0.1786
Files already downloaded and verified
Files already downloaded and verified
 ======> Training ResNet50_CIFAR100: SGD-128_lr=0.005_weightdecay=0.0001


Overall Progress: 100%|██████████| 20/20 [07:20<00:00, 22.04s/it, train_loss=3.8188, val_acc=18.84%, val_loss=3.7020]


Training complete!
Best Validation Accuracy: 0.1884
Files already downloaded and verified
Files already downloaded and verified
 ======> Training ResNet50_CIFAR100: SGD-128_lr=0.005_weightdecay=0.001


...

Training complete!
Best Validation Accuracy: 0.0898
Files already downloaded and verified
Files already downloaded and verified
 ======> Training ResNet50_CIFAR100: SGD-256_lr=0.001_weightdecay=0.0001


Overall Progress: 100%|██████████| 20/20 [06:32<00:00, 19.61s/it, train_loss=4.1877, val_acc=9.62%, val_loss=4.1134]


Training complete!
Best Validation Accuracy: 0.0962
Files already downloaded and verified
Files already downloaded and verified
 ======> Training ResNet50_CIFAR100: SGD-256_lr=0.001_weightdecay=0.001


Overall Progress: 100%|██████████| 20/20 [06:32<00:00, 19.60s/it, train_loss=4.1708, val_acc=9.63%, val_loss=4.1282]

Training complete!
Best Validation Accuracy: 0.0963


In [22]:
model_name = "SwinViT_T_CIFAR100"
num_epochs = 20
b_lr_wd = [
    (128, 0.005, 1e-5),
    (128, 0.005, 1e-4),
    (128, 0.005, 1e-3),
    (128, 0.001, 1e-3),
    (128, 0.001, 1e-4),
    (128, 0.001, 1e-5),
    (256, 0.005, 1e-5),
    (256, 0.005, 1e-4),
    (256, 0.005, 1e-3),
    (256, 0.001, 1e-5),
    (256, 0.001, 1e-4),
    (256, 0.001, 1e-3),
]
optimizer_name = 'Adam'

for batch_size, lr, weight_decay in b_lr_wd:
    trainloader_cutmix, testloader_cutmix = prepare_datasets(train_transform_basic, test_transform, cutmix_=1, batch_size=batch_size)
    model = SwinViT_T_CIFAR100()
    model = model.to(device)
    print(f" ======> Training {model_name}: {optimizer_name}-{batch_size}_lr={lr}_weightdecay={weight_decay}")
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.97)
    log_dir = f'./official_models_tuning/{model_name}_cutmix/{optimizer_name}-{batch_size}_lr{lr}_wd-{weight_decay}_e{num_epochs}'
    train_model(model, trainloader_cutmix, testloader_cutmix, optimizer, num_epochs, scheduler=scheduler, log_dir=log_dir)

Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: Adam-128_lr=0.005_weightdecay=1e-05


Overall Progress: 100%|██████████| 20/20 [06:33<00:00, 19.69s/it, train_loss=3.8504, val_acc=16.87%, val_loss=3.7502]


Training complete!
Best Validation Accuracy: 0.1704
Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: Adam-128_lr=0.005_weightdecay=0.0001


Overall Progress: 100%|██████████| 20/20 [06:35<00:00, 19.78s/it, train_loss=4.0520, val_acc=11.80%, val_loss=3.9698]


Training complete!
Best Validation Accuracy: 0.1210
Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: Adam-128_lr=0.005_weightdecay=0.001


...

Training complete!
Best Validation Accuracy: 0.2298
Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: Adam-256_lr=0.001_weightdecay=0.0001


Overall Progress: 100%|██████████| 20/20 [04:17<00:00, 12.87s/it, train_loss=3.7421, val_acc=20.27%, val_loss=3.6271]


Training complete!
Best Validation Accuracy: 0.2027
Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: Adam-256_lr=0.001_weightdecay=0.001


Overall Progress: 100%|██████████| 20/20 [04:17<00:00, 12.86s/it, train_loss=4.0224, val_acc=11.94%, val_loss=3.9782]

Training complete!
Best Validation Accuracy: 0.1194


change batch size

In [23]:
model_name = "ResNet50_CIFAR100"
num_epochs = 20
b_lr_wd = [
    (512, 0.005, 1e-4),
    (1024, 0.005, 1e-4),
]
optimizer_name = 'SGD'

for batch_size, lr, weight_decay in b_lr_wd:
    trainloader_cutmix, testloader_cutmix = prepare_datasets(train_transform_basic, test_transform, cutmix_=1, batch_size=batch_size)
    model = ResNet50_CIFAR100()
    model = model.to(device)
    print(f" ======> Training {model_name}: {optimizer_name}-{batch_size}_lr={lr}_weightdecay={weight_decay}")
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.97)
    log_dir = f'./official_models_tuning/{model_name}_cutmix/{optimizer_name}-{batch_size}_lr{lr}_wd-{weight_decay}_e{num_epochs}'
    train_model(model, trainloader_cutmix, testloader_cutmix, optimizer, num_epochs, scheduler=scheduler, log_dir=log_dir)

Files already downloaded and verified
Files already downloaded and verified
 ======> Training ResNet50_CIFAR100: SGD-512_lr=0.005_weightdecay=0.0001


Overall Progress: 100%|██████████| 20/20 [06:07<00:00, 18.35s/it, train_loss=4.0131, val_acc=11.76%, val_loss=3.9988]


Training complete!
Best Validation Accuracy: 0.1363
Files already downloaded and verified
Files already downloaded and verified
 ======> Training ResNet50_CIFAR100: SGD-1024_lr=0.005_weightdecay=0.0001


Overall Progress: 100%|██████████| 20/20 [06:13<00:00, 18.65s/it, train_loss=4.1852, val_acc=11.35%, val_loss=4.0276]

Training complete!
Best Validation Accuracy: 0.1135


In [24]:
model_name = "SwinViT_T_CIFAR100"
num_epochs = 20
b_lr_wd = [
    (512, 0.001, 1e-5),
    (1024, 0.001, 1e-5),
]
optimizer_name = 'Adam'

for batch_size, lr, weight_decay in b_lr_wd:
    trainloader_cutmix, testloader_cutmix = prepare_datasets(train_transform_basic, test_transform, cutmix_=1, batch_size=batch_size)
    model = SwinViT_T_CIFAR100()
    model = model.to(device)
    print(f" ======> Training {model_name}: {optimizer_name}-{batch_size}_lr={lr}_weightdecay={weight_decay}")
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.97)
    log_dir = f'./official_models_tuning/{model_name}_cutmix/{optimizer_name}-{batch_size}_lr{lr}_wd-{weight_decay}_e{num_epochs}'
    train_model(model, trainloader_cutmix, testloader_cutmix, optimizer, num_epochs, scheduler=scheduler, log_dir=log_dir)

Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: Adam-512_lr=0.001_weightdecay=1e-05


Overall Progress: 100%|██████████| 20/20 [03:51<00:00, 11.59s/it, train_loss=3.6287, val_acc=21.39%, val_loss=3.5375]


Training complete!
Best Validation Accuracy: 0.2140
Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: Adam-1024_lr=0.001_weightdecay=1e-05


Overall Progress:   0%|          | 0/20 [00:00<?, ?it/s]/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Overall Progress: 100%|██████████| 20/20 [03:47<00:00, 11.39s/it, train_loss=3.7537, val_acc=19.40%, val_loss=3.6858]

Training complete!
Best Validation Accuracy: 0.2082


Rest are some training records.

In [7]:
model_name = "SwinViT_T_CIFAR100"
num_epochs = 200
lr = 0.001
weight_decay = 1e-5
optimizer_name = 'AdamW'
batch_size = 128
# lr_schedules = ["StepLR", "CosineAnnealingLR", "ReduceLROnPlateau"]
lr_schedules = ["CosineAnnealingLR"]


for lr_schedule in lr_schedules:
    trainloader_cutmix, testloader_cutmix = prepare_datasets(train_transform_basic, test_transform, cutmix_=1, batch_size=batch_size)
    model = SwinViT_T_CIFAR100()
    model = model.to(device)
    print(f" ======> Training {model_name}: {optimizer_name}-{batch_size}_lr={lr}_{lr_schedule}_weightdecay={weight_decay}")
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.97)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(trainloader_cutmix), eta_min=1e-6)
    log_dir = f'./official_models_tuning/{model_name}_cutmix/{optimizer_name}-{batch_size}_lr{lr}_{lr_schedule}_wd-{weight_decay}_e{num_epochs}'
    train_model(model, trainloader_cutmix, testloader_cutmix, optimizer, num_epochs, scheduler=scheduler, log_dir=log_dir)

Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: AdamW-128_lr=0.001_CosineAnnealingLR_weightdecay=1e-05


Overall Progress: 100%|██████████| 200/200 [1:22:46<00:00, 24.83s/it, train_loss=2.2727, val_acc=46.32%, val_loss=2.6705]

Training complete!
Best Validation Accuracy: 0.4634


In [19]:
model_name = "SwinViT_T_CIFAR100"
num_epochs = 300
lr = 0.001
weight_decay = 1e-5
optimizer_name = 'AdamW'
batch_size = 128
# lr_schedules = ["StepLR", "CosineAnnealingLR", "ReduceLROnPlateau"]
lr_schedules = ["CosineAnnealingLR"]


for lr_schedule in lr_schedules:
    trainloader_cutmix, testloader_cutmix = prepare_datasets(train_transform_basic, test_transform, cutmix_=1, batch_size=batch_size)
    model = SwinViT_T_CIFAR100()
    model = model.to(device)
    print(f" ======> Training {model_name}: {optimizer_name}-{batch_size}_lr={lr}_{lr_schedule}_weightdecay={weight_decay}")
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.97)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(trainloader_cutmix), eta_min=1e-6)
    log_dir = f'./official_models_tuning/{model_name}_cutmix/{optimizer_name}-{batch_size}_lr{lr}_{lr_schedule}_wd-{weight_decay}_e{num_epochs}'
    train_model(model, trainloader_cutmix, testloader_cutmix, optimizer, num_epochs, scheduler=scheduler, log_dir=log_dir)

Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: AdamW-128_lr=0.001_CosineAnnealingLR_weightdecay=1e-05


Overall Progress:   0%|          | 1/300 [00:25<2:05:49, 25.25s/it, train_loss=4.3955, val_acc=8.14%, val_loss=4.2611]

Saving...


Overall Progress:   1%|          | 2/300 [00:50<2:05:30, 25.27s/it, train_loss=4.3098, val_acc=8.15%, val_loss=4.2415]

Saving...


...

Saving...


Overall Progress: 100%|█████████▉| 299/300 [2:05:01<00:25, 25.19s/it, train_loss=2.5089, val_acc=47.39%, val_loss=3.0177]

Saving...


Overall Progress: 100%|██████████| 300/300 [2:05:27<00:00, 25.09s/it, train_loss=2.5881, val_acc=46.26%, val_loss=3.0105]

Training complete!
Best Validation Accuracy: 0.4739


In [5]:
model_name = "SwinViT_T_CIFAR100"
num_epochs = 300
lr = 0.001
weight_decay = 1e-5
optimizer_name = 'AdamW'
batch_size = 128
# lr_schedules = ["StepLR", "CosineAnnealingLR", "ReduceLROnPlateau"]
lr_schedules = ["CosineAnnealingLR"]


for lr_schedule in lr_schedules:
    trainloader_basic, testloader_basic = prepare_datasets(train_transform_basic, test_transform, cutmix_=0, batch_size=batch_size)
    model = SwinViT_T_CIFAR100()
    model = model.to(device)
    print(f" ======> Training {model_name}: {optimizer_name}-{batch_size}_lr={lr}_{lr_schedule}_weightdecay={weight_decay}")
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.97)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(trainloader_basic), eta_min=1e-6)
    log_dir = f'./official_models_tuning/{model_name}_basic/{optimizer_name}-{batch_size}_lr{lr}_{lr_schedule}_wd-{weight_decay}_e{num_epochs}'
    train_model(model, trainloader_basic, testloader_basic, optimizer, num_epochs, scheduler=scheduler, log_dir=log_dir)

Files already downloaded and verified
Files already downloaded and verified
 ======> Training SwinViT_T_CIFAR100: AdamW-128_lr=0.001_CosineAnnealingLR_weightdecay=1e-05


Overall Progress:   0%|          | 1/300 [00:26<2:12:33, 26.60s/it, train_loss=4.1455, val_acc=13.27%, val_loss=3.8608]

Saving...


Overall Progress:   1%|          | 2/300 [00:51<2:06:48, 25.53s/it, train_loss=3.8934, val_acc=16.74%, val_loss=3.7033]

Saving...


...

Saving...


Overall Progress:  98%|█████████▊| 294/300 [2:02:19<02:30, 25.02s/it, train_loss=1.2362, val_acc=53.82%, val_loss=2.5000]

Saving...


Overall Progress: 100%|██████████| 300/300 [2:04:49<00:00, 24.96s/it, train_loss=1.2290, val_acc=52.97%, val_loss=2.5184]

Training complete!
Best Validation Accuracy: 0.5382
